In [69]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import numpy as np
import math
import torch
import os



from torch.autograd import Variable
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models

import matplotlib.pyplot as plt


input_dim = 784
epochs=10
train_size=20000
validation_size=2000
test_size=10000

In [70]:
path = "MNIST_Data/"

# declaring transform func i.e normalizing and resizing
train_transforms = transforms.Compose([transforms.Resize((28,28)),transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406],
                                                            [0.229, 0.224, 0.225])])

test_transforms = transforms.Compose([transforms.Resize((28,28)),transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406],
                                                           [0.229, 0.224, 0.225])])

# Loading dataset using ImageFolder method. Pass transforms in here.
train_data = datasets.ImageFolder(path + '/train', transform=train_transforms)
test_data = datasets.ImageFolder(path + '/test', transform=test_transforms)



In [71]:
print(train_data)


Dataset ImageFolder
    Number of datapoints: 60000
    Root location: MNIST_Data//train
    StandardTransform
Transform: Compose(
               Resize(size=(28, 28), interpolation=PIL.Image.BILINEAR)
               ToTensor()
               Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
           )


In [73]:
train_index = np.random.choice(len(train_data), train_size+validation_size, replace=False)
test_index = np.random.choice(len(test_data), test_size, replace = False)
train_index, validation_index = torch.utils.data.random_split(train_index,[train_size, validation_size])

training_data = torch.utils.data.Subset(dataset=train_data, indices=train_index)
validation_data = torch.utils.data.Subset(dataset=train_data, indices=validation_index)
testing_data = torch.utils.data.Subset(dataset=test_data, indices=test_index)
    

In [77]:
train_loader = torch.utils.data.DataLoader(training_data, batch_size=64, shuffle=True)
validation_loader = torch.utils.data.DataLoader(validation_data, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(testing_data, batch_size=64)

In [78]:
print(test_loader)

In [79]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        neurons_per_layer = [100,50,10]
        dropout=0.2
        self.ra1 = nn.Linear(input_dim*3, neurons_per_layer[0])
        self.ra2 = nn.Linear(neurons_per_layer[0], neurons_per_layer[1])
        self.ra3 = nn.Linear(neurons_per_layer[1], neurons_per_layer[2])
        self.dropout1 = nn.Dropout2d(dropout)
        self.dropout2 = nn.Dropout2d(dropout)
    def forward(self, x):
        x = F.relu(self.ra1(x))
        x = F.relu(self.ra2(x))
        x = self.ra3(x)
        return F.log_softmax(x,dim=1)
    

In [80]:
model = Net()
learning_rate=0.0005
epochs=10

# this statement tell our code if there gpu available on our machine or not.
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

Net(
  (ra1): Linear(in_features=2352, out_features=100, bias=True)
  (ra2): Linear(in_features=100, out_features=50, bias=True)
  (ra3): Linear(in_features=50, out_features=10, bias=True)
  (dropout1): Dropout2d(p=0.2, inplace=False)
  (dropout2): Dropout2d(p=0.2, inplace=False)
)

In [81]:
optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)
criterion = nn.CrossEntropyLoss()

In [ ]:
# Training

In [82]:
loss_function = nn.CrossEntropyLoss()
count =0

for epoch in range(epochs):
    trainAccuracy=0
    trainLoss=0
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data), Variable(target)
        data = data.view(-1, input_dim*3)
        optimizer.zero_grad()
        net_out = model(data)
    #     print('Targets',target)
        loss = loss_function(net_out, target)  # loss_func is criterion
        loss.backward()
        optimizer.step()
        trainAccuracy += (torch.argmax(net_out, dim=1) == target).float().mean().item()
        trainLoss += loss.item()
        count+=1
    totalTrainLoss=[]
    totalTrainAccuracy=[]
    totalTrainLoss.append(trainLoss/len(train_loader))
    totalTrainAccuracy.append(trainAccuracy/len(train_loader))
print(count)


3130


In [ ]:
# Validating

In [84]:
Accuracy=0
Loss=0
for batch_idx, (data, target) in enumerate(validation_loader):
    data, target = Variable(data), Variable(target)
    data = data.view(-1, input_dim*3)
    optimizer.zero_grad()
    net_out = model(data)
    loss = loss_function(net_out, target)  # loss_func is criterion
    Accuracy += (torch.argmax(net_out, dim=1) == target).float().mean().item()
    Loss += loss.item()
    count+=1
totalValidationLoss=Loss/len(validation_loader)
totalValidationAccuracy=Accuracy/len(validation_loader)


In [85]:
# testing

In [86]:

Accuracy=0
Loss=0
for batch_idx, (data, target) in enumerate(test_loader):
    data, target = Variable(data), Variable(target)
    data = data.view(-1, input_dim*3)
    optimizer.zero_grad()
    net_out = model(data)
    loss = loss_function(net_out, target)  # loss_func is criterion
    Accuracy += (torch.argmax(net_out, dim=1) == target).float().mean().item()
    Loss += loss.item()
    count+=1
totalTestLoss=Loss/len(test_loader)
totalTestAccuracy=Accuracy/len(test_loader)
print(count)

3319


In [87]:
torch.save(model.state_dict(), "chakkidumba.pth")

In [88]:
model.load_state_dict(torch.load("chakkidumba.pth"))

<All keys matched successfully>

In [89]:
print(len(totalTrainAccuracy))

1


In [90]:
print((totalTrainAccuracy))

[0.9329073482428115]


In [91]:
print(totalValidationAccuracy)

0.92333984375


In [92]:
print((totalTestAccuracy))

0.9308320063694268
